# Trabajo final

1. Limpiar la tabla extraida de la web
    - Limpiar las columnas y estandarizarlas a criterio suyo
    - Eliminar columnas no necesarias o nulas
    - Verificar el tipo de las columnas y asignar el correcto tipo
    - Guardarlo en un archivo de tipo parquet llamado: "indicadores_departamentales.parquet"
    - [OPCIONAL] Crear una base de datos sqlite con sqlalchemy y guardar lo antes limpiado como una tabla
2. Calcular cuantas personas hay en un km2 de la superficie de los departamentos : Población censal / Superficie
3. Que tan pequeño es la superficie del Callao, Lima Metropolitana,Lima Provincias con respecto al departamento de Amazonas: suma de superficies de los tres departamentos / superficie de amazonas
4. En cuanto a crecido la población según la estimación del 2022 con respecto a la población en el 2017

#### 1. Limpiar los datos

In [ ]:
import pandas as pd
import unicodedata
import re

In [ ]:
def clean_string(txt:str) -> str :
    """
    Normalizamos los textos
    """
    txt = str(txt).strip()
    txt = unicodedata.normalize('NFKD', txt).encode('ASCII', 'ignore').decode()

    txt = re.sub('[^0-9a-zA-Z&_]+', ' ', txt)
    txt = re.sub(' +', ' ',txt)
    return txt

In [ ]:
# Almacenamos los datos extraídos de la web
df = pd.read_html("https://es.wikipedia.org/wiki/Departamentos_del_Per%C3%BA")[1]

# Deseo quedarme solo con la segunda columna de nombres del df.columns (es un MultiIndex)
df.columns = df.columns.get_level_values(1)

# Observamos los datos para ver qué columnas eliminar
df.head()

# Eliminamos columnas que no necesitaremos como Bandera,Capital, Escudo, Creacion Ubicación,Estimación 2022[46]​.1
df.drop(columns=['Ubigeo','Bandera', 'Escudo','Capital', 'Creación','Ubicación','Estimación 2022[46]​.1'],inplace=True)

# Eliminamos la fila 26 ya que no es un departamento
df.drop(26,inplace=True)

# Renombramos las columnas
new_columns = {
    'Departamento': 'Departamento',
    'Censo 2017[49]​': 'Censo_2017',
    'Estimación 2022[46]​': 'Estimacion_2022',
    'Superficie[48]​ (km²)': 'Superficie(km²)'
}

df.rename(columns = new_columns,inplace=True)


# Cambiamos tipo de dato de las columnas

df['Departamento'] = df['Departamento'].astype('string')

# Cambiamos el valor Callao[nota 1] por solo Callao
df.loc[6,'Departamento'] = "Callao"

# Usamos la función clean_string para estandarozar los nombres de los departamentos
df['Departamento'] = df['Departamento'].apply(lambda x: clean_string(x).title())

df['Censo_2017'] = df['Censo_2017'].astype('string').str.replace("\xa0","").astype(int)
df['Estimacion_2022'] = df['Estimacion_2022'].astype('string').str.replace("\xa0","").astype(int)
df['Superficie(km²)'] = df['Superficie(km²)'].astype('string').str.replace("\xa0","").astype(int)


In [ ]:
df.to_parquet("indicadores_departamentales.parquet")

#### 2. Densidad Poblacional

In [ ]:
# Seleccionamos solo las columnas que nos interesan 
temp_df = df[['Departamento','Censo_2017','Superficie(km²)']]

# Dividimos población sobre superficie y redondeamos a 2
temp_df['Densidad_poblacional'] = round(temp_df['Censo_2017']/temp_df['Superficie(km²)'],2)

# Seleccionamos la densidad poblacional de cada departamento
temp_df[['Departamento','Densidad_poblacional']]

#### 3. Comparación de Superficies

In [ ]:
# Seleccionamos los índices de los departmentos
index_1 = df[df['Departamento'].isin(['Callao','Lima Metropolitana','Lima Provincias'])].index
index_2 = df[df['Departamento'].isin(['Amazonas'])].index

# Dividimos la suma de las superficies de 3 departamentos ('Callao','Lima Metropolitana','Lima Provincias') y dividimos sobre la superficie de Amazonas
diff = -df.loc[index_1,'Superficie(km²)'].sum()  + df.loc[index_2,'Superficie(km²)'].sum()
multi = round(df.loc[index_1,'Superficie(km²)'].sum() / df.loc[index_2,'Superficie(km²)'].sum(),2)

print(f"La suma de superficies de Callao, Lima Metropolitana, Lima Provincias es {multi} veces la superficie de Amazonas ({diff} km menos)")

#### 4. Comparación de poblaciones

In [ ]:
# Medimos el aumento de la población y hallamos el porcentaje
diff = df['Estimacion_2022'].sum() - df['Censo_2017'].sum()
aumento = round((df['Estimacion_2022'].sum() - df['Censo_2017'].sum())*100/ df['Censo_2017'].sum(),2)

print(f"La estimación poblacional para el 2022 aumenta en un {aumento} % ({diff}) comparado con el 2017 ")